Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Tutorial #1: Train an image classification model with Azure Machine Learning

In this tutorial, you train a machine learning model on remote compute resources. You'll use the training and deployment workflow for Azure Machine Learning service (preview) in a Python Jupyter notebook.  You can then use the notebook as a template to train your own machine learning model with your own data. This tutorial is **part one of a two-part tutorial series**.  

This tutorial trains a simple logistic regression using the [MNIST](https://docs.microsoft.com/azure/open-datasets/dataset-mnist) dataset and [scikit-learn](http://scikit-learn.org) with Azure Machine Learning.  MNIST is a popular dataset consisting of 70,000 grayscale images. Each image is a handwritten digit of 28x28 pixels, representing a number from 0 to 9. The goal is to create a multi-class classifier to identify the digit a given image represents. 

Learn how to:

> * Set up your development environment
> * Access and examine the data
> * Train a simple logistic regression model on a remote cluster
> * Review training results, find and register the best model

You'll learn how to select a model and deploy it in [part two of this tutorial](deploy-models.ipynb) later. 

## Prerequisites

See prerequisites in the [Azure Machine Learning documentation](https://docs.microsoft.com/azure/machine-learning/service/tutorial-train-models-with-aml#prerequisites).

On the computer running this notebook, conda install matplotlib, numpy, scikit-learn=0.22.1

## Set up your development environment

All the setup for your development work can be accomplished in a Python notebook.  Setup includes:

* Importing Python packages
* Connecting to a workspace to enable communication between your local computer and remote resources
* Creating an experiment to track all your runs
* Creating a remote compute target to use for training

### Import packages

Import Python packages you need in this session. Also display the Azure Machine Learning SDK version.

In [37]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.56.0


### Connect to workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `ws`.

In [38]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

aml-silo-dev-003	westeurope	rg-silo-dev-003


### Create experiment

Create an experiment to track the runs in your workspace. A workspace can have muliple experiments. 

In [39]:
experiment_name = 'cyto4try'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [40]:
import os
# os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpu-cluster")

In [41]:
os.environ

environ{'HBI_WORKSPACE': 'false',
        'JUPYTER_SERVER_ROOT': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cytox/code',
        'USER': 'azureuser',
        'MKL_THREADING_LAYER': 'GNU',
        'CI_WORKSPACE': 'aml-silo-dev-003',
        'SHLVL': '2',
        'LD_LIBRARY_PATH': '/anaconda/envs/azureml_py38_PT_TF/lib/',
        'HOME': '/home/azureuser',
        'CONDA_SHLVL': '2',
        'OLDPWD': '/home/azureuser/.vscode-server/bin/dc96b837cf6bb4af9cd736aa3af08cf8279f7685',
        'OBO_ENDPOINT': 'http://127.0.0.1:46808/OBO/token',
        'CI_LOCAL_UBUNTU_USER': 'azureuser',
        'AZURE_EXTENSION_DIR': '/opt/az/extensions',
        '_CE_M': '',
        'PYXTERM_DIMENSIONS': '80x25',
        'DEFAULT_IDENTITY_CLIENT_ID': '37c18f62-4303-4624-9b35-de6d658c0197',
        'LOGNAME': 'azureuser',
        'JOURNAL_STREAM': '8:59456',
        '_': '/anaconda/envs/azureml_py38_PT_TF/bin/python',
        'COLUMNS': '1000',
        'TERM': 'xterm-color',
        'AML_CloudName': 'A

In [42]:
ws.compute_targets['cytox']

Name,Workspace,State,Location,VmSize,Application URI,SSH,Docs
cytox,aml-silo-dev-003,Running,westeurope,Standard_NC4as_T4_v3,Jupyter JupyterLab,azureuser@51.145.150.245 -p 50000,Doc


### Create or Attach existing compute resource

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. You will submit Python code to run on this VM later in the tutorial. 
The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

In [43]:
from azureml.core.compute import AmlCompute, ComputeInstance
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = 'cytox' #os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 2)

# This example uses CPU VM. For using GPU VM, set SKU to Standard_NC6s_v3
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "Standard_NC4as_T4_v3")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is ComputeInstance:
        print("found compute target: " + compute_name)
# else:
#     print("creating new compute target...")
#     provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
#                                                                 min_nodes = compute_min_nodes, 
#                                                                 max_nodes = compute_max_nodes)

#     # create the cluster
#     compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
#     # can poll for a minimum number of nodes and for a specific timeout. 
#     # if no min node count is provided it will use the scale settings for the cluster
#     compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
#      # For a more detailed view of current AmlCompute status, use get_status()
#     print(compute_target.get_status().serialize())

found compute target: cytox


In [44]:
ws.compute_targets

{'cytox': {
   "id": "/subscriptions/25130c3f-778b-4637-bfb8-3b1b885b45e7/resourceGroups/rg-silo-dev-003/providers/Microsoft.MachineLearningServices/workspaces/aml-silo-dev-003/computes/cytox",
   "name": "cytox",
   "location": "westeurope",
   "tags": {
     "cytox-vm": "T4"
   },
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "westeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "Standard_NC4as_T4_v3",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://cytox.westeurope.instances.azureml.ms/tree/"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://cytox.westeurope.instances.azureml.ms/lab"
         }
       ],
       "connectivityEndpoints": {
         "publicIpAddress": "51.145.150.245",
         "privateIpAddress": "10.0.0.4"

You now have the necessary packages and compute resources to train a model in the cloud. 

## Explore data

Before you train a model, you need to understand the data that you are using to train it. In this section you learn how to:

* Download the MNIST dataset
* Display some sample images

### Download the MNIST dataset

Use Azure Open Datasets to get the raw MNIST data files. [Azure Open Datasets](https://docs.microsoft.com/azure/open-datasets/overview-what-are-open-datasets) are curated public datasets that you can use to add scenario-specific features to machine learning solutions for more accurate models. Each dataset has a corrseponding class, `MNIST` in this case, to retrieve the data in different ways.

This code retrieves the data as a `FileDataset` object, which is a subclass of `Dataset`. A `FileDataset` references single or multiple files of any format in your datastores or public urls. The class provides you with the ability to download or mount the files to your compute by creating a reference to the data source location. Additionally, you register the Dataset to your workspace for easy retrieval during training.

Follow the [how-to](https://aka.ms/azureml/howto/createdatasets) to learn more about Datasets and their usage in the SDK.

In [45]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/cytox/code/workspace/cytotoxicity'

In [46]:
from azureml.core import Dataset
# from azureml.opendatasets import MNIST

# data_folder = './data/cytoData_v2'
ws.set_default_datastore('workspaceblobcompound')
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)
ds_paths = [(ds, 'reference_dataset_compounds_v2/')]
dataset = Dataset.File.from_files(path = ds_paths)

workspaceblobcompound AzureBlob cytotoxstorageaccount compounds-images


In [47]:
dataset

{
  "definition": "EnginelessDataflow:\n---\ntype: mltable\npaths:\n  - pattern: \"azureml://subscriptions/25130c3f-778b-4637-bfb8-3b1b885b45e7/resourcegroups/rg-silo-dev-003/workspaces/aml-silo-dev-003/datastores/workspaceblobcompound/paths/reference_dataset_compounds_v2/\"\nmetadata:\n  infer_column_types: \"False\"\n"
}

In [48]:
# mounted_path = data_folder
# mount_context = dataset.mount(mounted_path)
# # dataset.download(target_path=mounted_path, overwrite=True)
# mount_context.start()

# import os
# print(os.listdir(mounted_path))
# print (mounted_path)

### Display some sample images

Load the compressed files into `numpy` arrays. Then use `matplotlib` to plot 30 random images from the dataset with their labels above them. Note this step requires a `load_data` function that's included in an `utils.py` file. This file is included in the sample folder. Please make sure it is placed in the same folder as this notebook. The `load_data` function simply parses the compresse files into numpy arrays.

In [49]:
# from PIL import Image
# compound_list = os.listdir(data_folder)
# imagelist = [os.path.join(data_folder, compound_list[10], f) for f in os.listdir(os.path.join(data_folder, compound_list[10]))]
# len(imagelist)

In [50]:
# Image.open(imagelist[10])

## Train on a remote cluster

For this task, you submit the job to run on the remote training cluster you set up earlier.  To submit a job you:
* Create a directory
* Create a training script
* Create a script run configuration
* Submit the job 

### Create a directory

Create a directory to deliver the necessary code from your computer to the remote resource.

In [51]:
import os
script_folder = os.path.join(os.getcwd(), experiment_name)
os.makedirs(script_folder, exist_ok=True)

In [52]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/cytox/code/workspace/cytotoxicity'

### Create a training script

To submit the job to the cluster, first create a training script. Run the following code to create the training script called `train.py` in the directory you just created. 

Notice how the script gets data and saves models:

+ The training script reads an argument to find the directory containing the data.  When you submit the job later, you point to the dataset for this argument:
`parser.add_argument('--data-folder', type=str, dest='data_folder', help='data directory mounting point')`


+ The training script saves your model into a directory named outputs. <br/>
`joblib.dump(value=clf, filename='outputs/sklearn_mnist_model.pkl')`<br/>
Anything written in this directory is automatically uploaded into your workspace. You'll access your model from this directory later in the tutorial.

The file `utils.py` is referenced from the training script to load the dataset correctly.  Copy this script into the script folder so that it can be accessed along with the training script on the remote resource.

In [53]:
# import shutil
# shutil.copy('train_5folds_az_job.py', script_folder)
# shutil.copy('i3d.py', script_folder)
# shutil.copy('cyto_loader_local.py', script_folder)
# shutil.copytree('data/train_test_folds', os.path.join(script_folder, 'train_test_folds'))
# # shutil.copy('img-classification-part1-training.ipynb', script_folder)

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/cytox/code/workspace/cytotoxicity/cyto4try/train_test_folds'

Bad pipe message: %s [b'\x058\x9d\xddIk\xd9\xa4\xe6\xb0\xfdw\xb6\xec$\xe8\xe8p Do\xab[\x93\x15\x90\xf8\xd0\xa5\x9aR\x96|(\xda1i\xf1\xe2\x05\xb5(\xed\xd7s\x11\x9e\xb8\xd2\xff\xf0\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00', b'\x03\x04']
Bad pipe message: %s [b"\xfedO3\xd7{t8\xb1\xe0\xc7H\xfb\x8d66Kc\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\

### Configure the training job

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on. Configure the ScriptRunConfig by specifying:

* The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution. 
* The compute target.  In this case you will use the AmlCompute you created
* The training script name, train.py
* An environment that contains the libraries needed to run the script
* Arguments required from the training script. 

In this tutorial, the target is AmlCompute. All files in the script folder are uploaded into the cluster nodes for execution. The data_folder is set to use the dataset.

First, create the environment that contains: the scikit-learn library, azureml-dataset-runtime required for accessing the dataset, and azureml-defaults which contains the dependencies for logging metrics. The azureml-defaults also contains the dependencies required for deploying the model as a web service later in the part 2 of the tutorial.

Once the environment is defined, register it with the Workspace to re-use it in part 2 of the tutorial.

In [19]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

envname = 'AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu'
env = Environment.get(ws, envname)

# to install required packages
# env = Environment('tutorial-env')
# cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1', 'numpy==1.23'])

# env.python.conda_dependencies = cd

# # Register environment to re-use later
# env.register(workspace = ws)

Then, create the ScriptRunConfig by specifying the training script, compute target and environment.

In [20]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', dataset.as_mount()]

src = ScriptRunConfig(source_directory=os.getcwd(),
                      script='train_5folds_az_job.py', 
                      arguments=args,
                      compute_target=compute_target,
                      environment=env)

In [21]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/cytox/code/workspace/cytotoxicity/cyto3try'

### Submit the job to the cluster

Run the experiment by submitting the ScriptRunConfig object. And you can navigate to Azure portal to monitor the run.

In [22]:
run = exp.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
cyto3try,cyto3try_1715288445_7c7fd54a,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


Since the call is asynchronous, it returns a **Preparing** or **Running** state as soon as the job is started.

## Monitor a remote run

In total, the first run takes **approximately 10 minutes**. But for subsequent runs, as long as the dependencies in the Azure ML environment don't change, the same image is reused and hence the container start up time is much faster.

Here is what's happening while you wait:

- **Image creation**: A Docker image is created matching the Python environment specified by the Azure ML environment. The image is built and stored in the ACR (Azure Container Registry) associated with your workspace. Image creation and uploading takes **about 5 minutes**. 

  This stage happens once for each Python environment since the container is cached for subsequent runs.  During image creation, logs are streamed to the run history. You can monitor the image creation progress using these logs.

- **Scaling**: If the remote cluster requires more nodes to execute the run than currently available, additional nodes are added automatically. Scaling typically takes **about 5 minutes.**

- **Running**: In this stage, the necessary scripts and files are sent to the compute target, then data stores are mounted/copied, then the entry_script is run. While the job is running, stdout and the files in the ./logs directory are streamed to the run history. You can monitor the run's progress using these logs.

- **Post-Processing**: The ./outputs directory of the run is copied over to the run history in your workspace so you can access these results.


You can check the progress of a running job in multiple ways. This tutorial uses a Jupyter widget as well as a `wait_for_completion` method. 

### Jupyter widget

Watch the progress of the run with a Jupyter widget.  Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [23]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

By the way, if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

### Get log results upon completion

Model training happens in the background. You can use `wait_for_completion` to block and wait until the model has completed training before running more code. 

In [24]:
# specify show_output to True for a verbose log
run.wait_for_completion(show_output=True) 

RunId: cyto3try_1715288445_7c7fd54a
Web View: https://ml.azure.com/runs/cyto3try_1715288445_7c7fd54a?wsid=/subscriptions/25130c3f-778b-4637-bfb8-3b1b885b45e7/resourcegroups/rg-silo-dev-003/workspaces/aml-silo-dev-003&tid=423d823d-bd94-4a87-b831-357bda79da31

Streaming user_logs/std_log.txt

Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.08891558647155762 seconds
Traceback (most recent call last):
  File "train_5folds_az_job.py", line 12, in <module>
    from cyto_loader_local import cytoDataset, Normaliztion, ToTensor, Rescale, RandomHorizontalFlip, RandomCrop, CenterCrop #, worker_init_fn #, RandomHorizontalFlip2CHNL, Rescale2CHNL, RandomCrop2CHNL,Rescale2CHNL, CenterCrop2CHNL
  File "/mnt/azureml/cr/j/e368e02bb7da4e269913c81435d99599/exe/wd/cyto_loader_local.py", line 11, in <module>
    import cv2
ModuleNotFoundError: No module named 'cv2'


Execution Summary
RunId: cyto3try_1715288445_7c7fd54a
Web View: https://ml.azure.com/r

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process 'python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"train_5folds_az_job.py\", line 12, in <module>\n    from cyto_loader_local import cytoDataset, Normaliztion, ToTensor, Rescale, RandomHorizontalFlip, RandomCrop, CenterCrop #, worker_init_fn #, RandomHorizontalFlip2CHNL, Rescale2CHNL, RandomCrop2CHNL,Rescale2CHNL, CenterCrop2CHNL\n  File \"/mnt/azureml/cr/j/e368e02bb7da4e269913c81435d99599/exe/wd/cyto_loader_local.py\", line 11, in <module>\n    import cv2\nModuleNotFoundError: No module named 'cv2'\n\n",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z",
    "componentName": "CommonRuntime"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message":

### Display run results

You now have a model trained on a remote cluster.  Retrieve all the metrics logged during the run, including the accuracy of the model:

In [ ]:
print(run.get_metrics())

In the next tutorial you will explore this model in more detail.

## Register model

The last step in the training script wrote the file `outputs/sklearn_mnist_model.pkl` in a directory named `outputs` in the VM of the cluster where the job is executed. `outputs` is a special directory in that all content in this  directory is automatically uploaded to your workspace.  This content appears in the run record in the experiment under your workspace. Hence, the model file is now also available in your workspace.

You can see files associated with that run.

In [ ]:
print(run.get_file_names())

Register the model in the workspace so that you (or other collaborators) can later query, examine, and deploy this model.

In [ ]:
# register model 
model = run.register_model(model_name='3d_cytox', model_path='outputs/3d_cytox.pkl')
print(model.name, model.id, model.version, sep='\t')